### Time-Independent One-Dimensional (1-D) Schrödinger Equation 
###### $ -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} + V(x)\psi = H\psi = E\psi $ ( $ Ax = \lambda x $ )

In [1]:
import numpy as np
import numpy.linalg as lin
import math
import ipywidgets as widgets
from ipywidgets import interact_manual
from IPython.display import HTML, display
import plotly.graph_objects as go
import plotly.express as px
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.cm as cm   # for color variations

# Set numpy print options for debugging (optional)
np.set_printoptions(threshold=784, linewidth=np.inf)

###############################################################################
# Global Constants and Unit Conversions
###############################################################################
bohr_radius_ang = physical_constants['Bohr radius'][0] * 1e10  # in Angstrom
ang2bohr = 1 / bohr_radius_ang  # ~1.88973
har2ev = physical_constants['Hartree energy'][0] / physical_constants['electron volt'][0]  # ~27.21140795
laplacianDim = 3

###############################################################################
# Laplacian Functions (Finite Difference Method)
###############################################################################
def laplacianCoeff(laplacianDim): 
    a = np.zeros((2*laplacianDim+1, 2*laplacianDim+1))
    c = np.zeros(2*laplacianDim+1)
    c[2] = math.factorial(2)
    for i in range(2*laplacianDim+1):
        for j in range(2*laplacianDim+1):
            a[i,j] = (j - laplacianDim)**i
    inva = lin.inv(a)
    b = np.matmul(inva, c)
    lcoeff = b[laplacianDim:2*laplacianDim+1]
    return lcoeff

def laplacian(ngridx, laplacianDim, dx):
    lcoeff = laplacianCoeff(laplacianDim)
    loprt = np.zeros((ngridx, ngridx))
    for i in range(ngridx):
        for j in range(-laplacianDim, laplacianDim+1):
            k = i + j
            if k >= ngridx:
                k -= ngridx
            elif k < 0:
                k += ngridx
            loprt[i][k] = lcoeff[abs(j)] / (dx**2)
    return loprt

###############################################################################
# Potential and Hamiltonian Functions
###############################################################################
def potential(ngridx, pot_shape=0, pot_height_eV=25, curvature=0.5, width=0, slope=0.5):
    pot_height_har = pot_height_eV / har2ev
    pot_grid = np.zeros(ngridx)
    pot_grid[0] = 1e9
    pot_grid[ngridx-1] = 1e9

    if pot_shape == 0:      # Finite square well
        pot_grid[1:ngridx] = pot_height_har
        pot_grid[int(ngridx * (0.5 - width)) : int(ngridx * (0.5 + width))] = 0
    elif pot_shape == 1:    # infinite square well
        pot_grid[1:ngridx] = 1e9
        pot_grid[int(ngridx * (0.5 - width)) : int(ngridx * (0.5 + width))] = 0        
    elif pot_shape == 2:    # Symmetric double square well
        pot_grid[1:ngridx] = pot_height_har
        pot_grid[int(ngridx * (0.5 - width/2 - width)) : int(ngridx * (0.5 - width/2 ))] = 0
        pot_grid[int(ngridx * (0.5 + width/2 )) : int(ngridx * (0.5 + width/2 + width))] = 0
    elif pot_shape == 3:    # Harmonic
        for i in range(1, ngridx):
            pot_grid[i] = (i - (ngridx-1)//2)**2
        pot_grid[1:ngridx] = pot_grid[1:ngridx] * pot_height_har / (((ngridx-1)//2)**2) / curvature * 30
    elif pot_shape == 4:    # Triangular
        pot_grid[:] = 1e6
        for i in range(int(ngridx*0.4), ngridx-1):
            pot_grid[i] = (pot_height_har * abs(i - ngridx*0.4) / (ngridx*0.6)) * slope

        
    return pot_grid

def hamiltonian(ngridx, laplacianDim, dx, pot_height_eV=25, pot_shape=0, curvature=0.5, width=0, slope=0.6):
    pot_op = np.zeros((ngridx, ngridx))
    pot_1d = potential(ngridx, pot_shape, pot_height_eV, curvature, width, slope)
    laplacian_op = laplacian(ngridx, laplacianDim, dx)
    np.fill_diagonal(pot_op, val=pot_1d)
    ham = -laplacian_op/2. + pot_op
    return ham

def time_inde_schr_eq(pot_shape=0, curvature=0.5, width=0, slope=0.6, pot_height_eV=25):
    print("\n(1) Run Simulation...")
    ham_op = hamiltonian(ngridx, laplacianDim, dx, pot_height_eV, pot_shape, curvature, width, slope)
    (eigval, eigvec) = np.linalg.eigh(ham_op)
    print("\nDone!!")
    return eigval, eigvec


def compute_energy_components(eigval, eigvec, numOFstate, pot_shape, curvature, width, slope, pot_height_eV):
    # T_op = -laplacian/2, V_op = diag(potential)
    T_op = -laplacian(ngridx, laplacianDim, dx) / 2.
    V_1d = potential(ngridx, pot_shape, pot_height_eV, curvature, width, slope)
    V_op = np.diag(V_1d)

    pot = potential(ngridx, pot_shape, pot_height_eV, curvature, width, slope) * har2ev
    filtered_indices = [i for i in range(len(eigval)) if eigval[i] * har2ev < pot[650]]
    num_to_print = min(numOFstate, len(filtered_indices))
    print("\nEnergy components for filtered eigenstates (used in animation):")
    for idx in range(num_to_print):
        j = filtered_indices[idx]
        psi = eigvec[:, j]
        T_exp = np.vdot(psi, T_op @ psi).real
        V_exp = np.vdot(psi, V_op @ psi).real
        total = T_exp + V_exp
        print(f"State {j+1}: Kinetic = {T_exp * har2ev:.2f} eV, Potential = {V_exp * har2ev:.2f} eV, Total = {total * har2ev:.2f} eV")

# Animation Function: Animate the time evolution of multiple eigenstates' real, imaginary, and probability density
# Only animate eigenstates with eigenenergy below the potential height.
def animate_eigenstate(eigval, eigvec, numOFstate, boxL, ngridx, pot_shape, pot_height_eV, curvature, width, slope, y_max, sim_time):

    print("\n(2) Visualizing ...")
    print("\tRed \t= Real part of Ψ \n\tBlue \t= Imaginary part of Ψ \n\tGrey shaded = Probability Ψ*Ψ")
    # Define the x-axis in Angstrom units
    x = np.linspace(-boxL/2, boxL/2, ngridx) / ang2bohr
    pot = potential(ngridx, pot_shape, pot_height_eV, curvature, width, slope) * har2ev
    height = 2  # eV, coefficient for visualization

    # Filter eigenstates with eigenenergy below the potential height
    filtered_indices = [i for i in range(len(eigval)) if eigval[i] * har2ev < pot[650]]
    
    num_to_plot = min(numOFstate, len(filtered_indices))
    if num_to_plot == len(filtered_indices):
        print("\n*** Draw the eigenstates lower than the height of the potential well")
    
    # Compute normalization and energy offsets for each filtered eigenstate
    len_psi_list = [np.sum(np.abs(eigvec[:, j])**2) for j in filtered_indices[:num_to_plot]]
    E_eV_list = [eigval[j] * har2ev for j in filtered_indices[:num_to_plot]]
    E_J_list  = [eigval[j] * physical_constants["Hartree energy"][0] for j in filtered_indices[:num_to_plot]]

    
    
    hbar = physical_constants["reduced Planck constant"][0]
    sim_time_s = sim_time * 1e-15  # Convert fsec to seconds
    time_step = 1e-17             # 0.01 fsec
    t_values = np.arange(0, sim_time_s + time_step, time_step)
    
    fig, ax = plt.subplots()
    ax.plot(x, pot, color='black', linewidth=1.2)
    
    # Generate state-specific color variations within desired colormaps.
    if num_to_plot > 1:
        real_colors = [cm.autumn(0.0 + 0.2*(i/(num_to_plot-1))) for i in range(num_to_plot)]
        imag_colors = [cm.winter(0.0 + 0.2*(i/(num_to_plot-1))) for i in range(num_to_plot)]
        fill_colors = [cm.Greys(0.4 + 0.2*(i/(num_to_plot-1))) for i in range(num_to_plot)]
    else:
        real_colors = [cm.autumn(0.0)]
        imag_colors = [cm.cool(0.4)]
        fill_colors = [cm.Greys(0.6)]
    
    # Create line objects for real and imaginary parts using state-specific colors.
    lines_real = []
    lines_imag = []
    for idx in range(num_to_plot):
        line_r, = ax.plot([], [], color=real_colors[idx], linestyle='-')
        line_i, = ax.plot([], [], color=imag_colors[idx], linestyle='-')
        lines_real.append(line_r)
        lines_imag.append(line_i)
    
    # Create initial fill for probability region for each eigenstate (initially zero area) using state-specific fill colors.
    fill_poly = []
    for idx in range(num_to_plot):
        poly = ax.fill_between(x, np.full_like(x, E_eV_list[idx]),
                               np.full_like(x, E_eV_list[idx]), color=fill_colors[idx], alpha=0.5)
        fill_poly.append(poly)
    
    # Add time text annotation in the upper left corner (using Axes coordinates)
    time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
    
    # Create eigenvalue text annotations for each eigenstate.
    # They are placed at a fixed x position (e.g., x = 8 Angstrom) and at y = E_eV_list[idx] (plus a small offset)
    eigen_text = []
    # y_max = E_eV_list
    x_text = 8   # in Angstrom units
    for idx in range(num_to_plot):
        if( E_eV_list[idx] + y_max/40) < y_max:
            txt = ax.text(x_text, E_eV_list[idx] + y_max/40, f"{E_eV_list[idx]:.2f} eV",
                        color='black', fontsize=10, verticalalignment='center')
            eigen_text.append(txt)
    
    ax.set_xlim(-15, 15)
    ax.set_ylim(-0.125*y_max, y_max)
    ax.set_xlabel("Box [Angstrom]")
    ax.set_ylabel("Energy [eV]")
    ax.set_title("Time Evolution of Eigenstates (Real, Imaginary, Probability Density)")
    
    def init():
        for line in lines_real + lines_imag:
            line.set_data([], [])
        for idx in range(num_to_plot):
            fill_poly[idx].remove()
            fill_poly[idx] = ax.fill_between(x, np.full_like(x, E_eV_list[idx]),
                                             np.full_like(x, E_eV_list[idx]), color=fill_colors[idx], alpha=0.5)
        time_text.set_text('')
        return lines_real + lines_imag + fill_poly + [time_text] + eigen_text
    
    def animate(i):
        t = t_values[i]
        artists = []
        for idx, j in enumerate(filtered_indices[:num_to_plot]):
            phase = np.exp(-1j * E_J_list[idx] * t / hbar)
            psi_t = eigvec[:, j] * phase
            real_scaled = psi_t.real / len_psi_list[idx] * height + E_eV_list[idx]
            imag_scaled = psi_t.imag / len_psi_list[idx] * height + E_eV_list[idx]
            # Compute probability curve using the normalization for |Ψ| scaled above the eigenvalue
            # prob_scaled = (real_scaled - E_eV_list[idx])**2 + (imag_scaled - E_eV_list[idx])**2 + E_eV_list[idx]
            prob_scaled = ((psi_t.real)**2 + (psi_t.imag)**2 ) / len_psi_list[idx] * height * 6 + E_eV_list[idx]
            lines_real[idx].set_data(x, real_scaled)
            lines_imag[idx].set_data(x, imag_scaled)
            fill_poly[idx].remove()
            fill_poly[idx] = ax.fill_between(x, np.full_like(x, E_eV_list[idx]), prob_scaled,
                                             color=fill_colors[idx], alpha=0.5)
            artists.extend([lines_real[idx], lines_imag[idx], fill_poly[idx]])
        time_text.set_text(f"Time: {t*1e15:.2f} fsec")
        artists.append(time_text)
        artists.extend(eigen_text)
        return artists
    
    ani = FuncAnimation(fig, animate, frames=len(t_values), init_func=init, interval=50, blit=False)
    
    print("\n")
    print("Done!!")
    
    display(HTML(ani.to_jshtml()))

###############################################################################
# Interactive Widget
###############################################################################
if __name__=="__main__":
    boxL_ang = 100        # Box Length in Angstrom
    ngridx = 1001         # Number of grid points
    laplacianDim = 3
    boxL = boxL_ang * ang2bohr
    dx = boxL / ngridx

    pot_shape_widget = widgets.Dropdown(
        options=[
            ('Finite Square Well', 0),
            ('Infinite Square Well', 1),
            ('Double Square Well', 2),
            ('Harmonic Oscillator', 3),
            ('Triangular Well', 4),
            
        ],
        value=0,
        description='Potential Shape: ',
        style={'description_width': 'initial'},
    )
    curvature_widget = widgets.FloatSlider(
        min=0.1, max=1, step=0.1,
        value=0.4,
        description='Curvature of harmonic well: narrow <--> wide',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    width_widget = widgets.FloatSlider(
        min=2, max=20, step=1,
        value=5,
        description='Width [Ang]',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    slope_widget = widgets.FloatSlider(
        min=0.1, max=5, step=0.1,
        value=3,
        description='Slope of triangular well: gentle <--> steep',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    pot_height_widget = widgets.FloatSlider(
        value=4, min=0.5, max=8, step=0.1, 
        description='Potential Height [eV]:',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    y_max_widget = widgets.FloatSlider(
        min=2, max=50, step=1,
        value=5,
        description='y range [eV]:',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    sim_time_widget = widgets.FloatSlider(
        min=0.1, max=10, step=0.1,
        value=0.1,
        description='Simulation Time [fsec]:',
        readout=True,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px', margin='15px 0px 15px 0px')
    )
    
    def update_visibility(change):
        value = pot_shape_widget.value
        curvature_widget.layout.display = 'block'   if value == 3 else 'none'
        width_widget.layout.display     = 'block'   if value <= 2 else 'none'
        pot_height_widget.layout.display = 'block'  if value == 0 or value == 2 else 'none'
        slope_widget.layout.display     = 'block'   if value == 4 else 'none'

    curvature_widget.layout.display = 'none'
    # width_widget.layout.display = 'none'
    # pot_height_widget.layout.display = 'none'
    slope_widget.layout.display = 'none'
    pot_shape_widget.observe(update_visibility, names='value')

    @interact_manual(
        numOFstate = widgets.IntSlider(min=1, max=10, step=1, value=5, description='# of States: '),
        pot_shape  = pot_shape_widget,
        curvature = curvature_widget,
        width     = width_widget,
        slope     = slope_widget,
        pot_height_eV = pot_height_widget,
        y_max     = y_max_widget,
        sim_time  = sim_time_widget
    )
    def interactive(numOFstate, pot_shape, curvature, width, slope, pot_height_eV, y_max, sim_time):
        (eigval, eigvec) = time_inde_schr_eq(pot_shape, curvature, width/200, slope, pot_height_eV) # dividiing 200 = change it to Anstrom

        compute_energy_components(eigval, eigvec, numOFstate, pot_shape, curvature, width/200, slope, pot_height_eV)
        animate_eigenstate(eigval, eigvec, numOFstate, boxL, ngridx, pot_shape, pot_height_eV, curvature, width/200, slope, y_max, sim_time)

    readme = r"""
    """
    print(readme)
    pass


interactive(children=(IntSlider(value=5, description='# of States: ', max=10, min=1), Dropdown(description='Po…

### 2025 Spring "EE211: Physical Electronics"
#### Minsu Jeong, KAIST Electrical Engineering

#### 1. Solving time-independent one-dimensional (1-D) Schrödinger equation using matrix diagonalization

###### Last updated : 2025. 03. 09

###### To solve:
 $ -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} + V(x)\psi = H\psi = E\psi $ ( $ Ax = \lambda x $ )
 
###### Numerical implementation:


1. Define Laplacian using FDM and external Potential for 1-D Hamiltonian.

2. Construct 1-D Hamiltonian using the information from step 1.

3. Diagonalize the Hamiltonian using numpy.eigh and get eigenvalues ($E_n$) and eigenvectors ($\psi$).

4. Visualize with interaction.
